In [1]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [2]:
S_data = pd.read_csv(r'C:\Data Analysis - Files\Project 5 - Superstore\Sample-Superstore.csv', sep=',', encoding='latin1', on_bad_lines='skip')

In [3]:
S_data = S_data.drop(columns=['Row ID'])
S_data = S_data.rename(columns={'Order ID': 'Order_ID', 'Customer ID': 'Customer_ID', 'Product ID': 'Product_ID'})

In [4]:
#Creating an empty sqlite database to populate with the newly created CSVs
conn = sqlite3.connect("superstore_database.db")

In [5]:
# Establishing a cursor object (a tool that executes SQL code against the database)
c = conn.cursor()

In [6]:
#create the (empty) tables which will be included in the database.

#Product_Details

c.execute('''
CREATE TABLE IF NOT EXISTS "Product_Details" (
	"Product_ID"	CHAR(20) NOT NULL,
	"Category"	VARCHAR(30) NOT NULL,
	"Sub-Category"	VARCHAR(30) NOT NULL,
    "Product Name"	VARCHAR(30) NOT NULL,
	PRIMARY KEY("Product_ID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Customer_Details" (
	"Customer_ID"	CHAR(30) NOT NULL,
	"Customer Name"	VARCHAR(30) NOT NULL,
	"Segment"	VARCHAR(15) NOT NULL,
	PRIMARY KEY("Customer_ID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Address_Details" (
	"Address_ID" INTEGER PRIMARY KEY AUTOINCREMENT,
  	"Country" text NOT NULL,
  	"City" text NOT NULL,
  	"State" text NOT NULL,
  	"Postal Code" integer NOT NULL,
  	"Region" text NOT NULL
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Order_Details" (
	"Order_ID"	CHAR(20) NOT NULL,
	"Order Date" date NOT NULL,
    "Ship Date" date NOT NULL, 
    "Ship Mode" varchar NOT NULL,
    "Customer_ID" varchar NOT NULL,
	PRIMARY KEY("Order_ID"),
	FOREIGN KEY(Customer_ID) REFERENCES Customer_Details(Customer_ID)
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Product_Sales" (
	"Sales" integer NOT NULL,
    "Quantity"  integer NOT NULL,
    "Discount"  integer NOT NULL,
    "Profit"  integer NOT NULL,
    "Order_ID" varchar NOT NULL,
    "Product_ID" varchar NOT NULL,
    FOREIGN KEY(Order_ID) REFERENCES Order_Details(Order_ID),
    FOREIGN KEY(Product_ID) REFERENCES Product_Details(Product_ID)
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Customer_address_junction" (
    Customer_ID INTEGER,
    Address_ID INTEGER,
    PRIMARY KEY (Customer_ID, Address_ID),
    FOREIGN KEY (Customer_ID) REFERENCES Customer_Details(Customer_ID),
    FOREIGN KEY (Address_ID) REFERENCES Address_Details(Address_ID)
);
''')

In [7]:
# If the table already exists and you want to append data:
S_data_order = S_data [["Order_ID","Order Date","Ship Date","Ship Mode","Customer_ID"]]
S_data_order = S_data_order.drop_duplicates(subset=['Order_ID'])
S_data_order.to_sql('Order_Details', conn, if_exists='append', index=False)

S_data_Customer = S_data [["Customer_ID","Customer Name","Segment"]]
S_data_Customer = S_data_Customer.drop_duplicates(subset=['Customer_ID'])
S_data_Customer.to_sql('Customer_Details', conn, if_exists='append', index=False)

S_data_Address = S_data [["Country","City",	"State","Postal Code", "Region"]]
S_data_Address = S_data_Address.drop_duplicates()
#S_data_Address.info()
S_data_Address.to_sql('Address_Details', conn, if_exists='append', index=False)

S_data_Product = S_data [["Product_ID", "Category", "Sub-Category", "Product Name"]]
S_data_Product = S_data_Product.drop_duplicates(subset=['Product_ID'])
#S_data_Product.info()
S_data_Product.to_sql('Product_Details', conn, if_exists='append', index=False)

S_data_Product_Sales = S_data [["Sales", "Quantity", "Discount", "Profit", "Order_ID", "Product_ID"]]
S_data_Product_Sales = S_data_Product_Sales.drop_duplicates()
S_data_Product_Sales.info()
S_data_Product_Sales.to_sql('Product_Sales', conn, if_exists='append', index=False)

#Commit changes and close the connection properly
conn.commit()  # Commit changes
#conn.close()   # Then close the connection


<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9993
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sales       9993 non-null   float64
 1   Quantity    9993 non-null   int64  
 2   Discount    9993 non-null   float64
 3   Profit      9993 non-null   float64
 4   Order_ID    9993 non-null   object 
 5   Product_ID  9993 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 546.5+ KB


In [8]:
S_data_Address.info()
# Adding a sequential number column
S_data_Address['Address_ID'] = range(1, len(S_data_Address) + 1)
S_data_Address['Full Address'] = S_data['Country'] + ' ' + S_data['City'].fillna('') + ' ' + S_data['State'] + ' ' + S_data['Postal Code'].fillna('').astype(str) + ' ' + S_data['Region']
S_data_Address = S_data_Address.drop(["Country", "City", "State", "Postal Code", "Region"],axis=1)
S_data_Address.tail()

<class 'pandas.core.frame.DataFrame'>
Index: 632 entries, 0 to 9881
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      632 non-null    object
 1   City         632 non-null    object
 2   State        632 non-null    object
 3   Postal Code  632 non-null    int64 
 4   Region       632 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.6+ KB


,Address_ID,Full Address
9760,628,United States Springdale Arkansas 72762 South
9801,629,United States Lodi California 95240 West
9834,630,United States La Porte Texas 77571 Central
9868,631,United States Mason Ohio 45040 East
9881,632,United States Woodstock Georgia 30188 South


In [9]:
S_data['Full Address'] = S_data['Country'] + ' ' + S_data['City'].fillna('') + ' ' + S_data['State'] + ' ' + S_data['Postal Code'].fillna('').astype(str) + ' ' + S_data['Region']
S_data['Full Address with CustID'] = S_data['Customer_ID'] + ' ' + S_data['Country'] + ' ' + S_data['City'].fillna('') + ' ' + S_data['State'] + ' ' + S_data['Postal Code'].fillna('').astype(str) + ' ' + S_data['Region']
unique_Address_with_CustID = S_data[['Customer_ID','Full Address with CustID', 'Full Address']]
unique_Address_with_CustID_AddressID = pd.merge(unique_Address_with_CustID, S_data_Address, on='Full Address', how='left')
unique_Address_with_CustID_AddressID.info()

#S_data_Customer_Address = S_data['Customer_ID'] + ' ' + S_data['Full Address']
#S_data_Customer_Address.info()
#S_data_Customer_Address = S_data_Customer_Address.drop_duplicates()
#S_data_Customer_Address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Customer_ID               9994 non-null   object
 1   Full Address with CustID  9994 non-null   object
 2   Full Address              9994 non-null   object
 3   Address_ID                9994 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 312.4+ KB


In [10]:
unique_Address_with_CustID_AddressID = unique_Address_with_CustID_AddressID.drop_duplicates(subset=['Full Address with CustID'])
unique_Address_with_CustID_AddressID.info()
unique_Address_with_CustID_AddressID = unique_Address_with_CustID_AddressID.drop(['Full Address with CustID','Full Address'],axis=1)
unique_Address_with_CustID_AddressID.info()
#unique_Address_with_CustID_AddressID = unique_Address_with_CustID_AddressID ['Customer_ID','Address_ID']
#unique_Address_with_CustID_AddressID.to_sql('Customer_address_junction', conn, if_exists='append', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4910 entries, 0 to 9993
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Customer_ID               4910 non-null   object
 1   Full Address with CustID  4910 non-null   object
 2   Full Address              4910 non-null   object
 3   Address_ID                4910 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 191.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 4910 entries, 0 to 9993
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  4910 non-null   object
 1   Address_ID   4910 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 115.1+ KB


In [11]:

unique_Address_with_CustID_AddressID.to_sql('Customer_address_junction', conn, if_exists='append', index=False)
#Commit changes and close the connection properly
conn.commit()  # Commit changes
conn.close()   # Then close the connection

In [12]:
S_data['Full Address with CustID'] = S_data['Full Address with CustID'].drop_duplicates()
S_data['Full Address with CustID'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9994 entries, 0 to 9993
Series name: Full Address with CustID
Non-Null Count  Dtype 
--------------  ----- 
4910 non-null   object
dtypes: object(1)
memory usage: 78.2+ KB
